In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
import os
import errno
%matplotlib inline

## Function for the Time Series Forecast

In [2]:
#Functions
def autoArimaModelVar(dataModel,dataTotal,dataTotalCopy,counter1): #make predictions
    model = auto_arima(dataModel, start_p=1, start_q=1,
                               max_p=3, max_q=3, m=12,
                               start_P=0, seasonal=True,
                               d=1, D=1, trace=True,
                               error_action='ignore',  
                               suppress_warnings=True, 
                               stepwise=True)

    # Forecast
    n_periods = 6 # 1 hour
    index_of_fc = pd.date_range(dataTotal.index[-1], periods = n_periods, freq='10min')
    fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
    fc1=pd.DataFrame(fc,index=index_of_fc) 
    confint1=pd.DataFrame(confint,index=index_of_fc) 
    fc1.columns=['Predicted']
    confint1.columns=['LowerConfidenceInterval','UpperConfidenceInterval']

    # Store the forecast values in the dataFrame
    forecastValues=fc1 # predicted value
    forecastValues['LowerConfidenceInterval']=confint1['LowerConfidenceInterval'] #lower confidence interval
    forecastValues['UpperConfidenceInterval']=confint1['UpperConfidenceInterval'] #upper confidence interval

    # calculate new variables
    #forecastValues['error']=forecastValues['val01_ambient_temperature']-forecastValues['Predicted'] #when we have values
    forecastValues['std_dev']=forecastValues['UpperConfidenceInterval']-forecastValues['Predicted']
    forecastValues = forecastValues.loc[~forecastValues.index.duplicated(keep='first')] #indexes must be unique for concat
    
    # Create a column with the value of the measures
    valueCol=dataTotalCopy #  
    valueCol = valueCol.loc[~valueCol.index.duplicated(keep='first')] #indexes must be unique for concat
    
    # Concatenate using the index, the values and the forecast
    frames=[valueCol,forecastValues]
    result=pd.concat(frames,axis=1)
    #result=pd.join(frames)
    result.tail(10)

    # Add a column called signal with the name of the signal preparing for the long format, instead of wide.
    result['signal']=pd.DataFrame(dataTotal).columns[0]
    
    # Naming column "value" with all the measured values 
    result['value']=pd.DataFrame(valueCol)

    #delete all the NaN after the concatenate
    resultNoNaN = result.replace(np.nan, '', regex=True)

    return resultNoNaN

# Import the data and clean it

## Store the data and import new data
A file is used to store the historical data (old+new) for the model with predictions. Therefore it is necessary to create an empty file to store the historical data. And before creating, it is necessary to verify if the file already exists.

In [3]:
# Create initial file to store data
flags = os.O_CREAT | os.O_EXCL | os.O_WRONLY

try:
    file_handle = os.open('data1.csv', flags)
except OSError as e:
    if e.errno == errno.EEXIST:  # Failed as the file already exists.
        pass
    else:  # Something unexpected went wrong so reraise the exception.
        raise
else:  # No exception, so the file must have been created successfully.
    with os.fdopen(file_handle, 'w') as file_obj:
        # Using `os.fdopen` converts the handle to an object that acts like a
        # regular Python file object, and the `with` context manager means the
        # file will be automatically closed when we're done with it.
        file_obj.write("Initializing the table") 

# Import the new data, preferably only the new data (not exported before)
data=pd.read_excel('HIS-VAP (dat).xlsx',header=None) #header starts in row 4

# Import the cleaned data from the historical data file. 
data2=pd.read_csv("~/Documents/Kschool/00_TFM/data1.csv")

# Create column headers
col_header=data.loc[4].str[15:] #header starts in row four and we need from the character 15 onwards.
col_header[0]='measure_time'
col_header[1]='minute_counter'
col_header

# duplicate the data original data to prevent corrupting it
data1=data

# assign the column headers to the data
data1=data1.rename(columns=col_header)

# clean data & columns
data1=data1.drop([0,1,2,3,4,5,6,7,8,9,10]) #delete rows with no added value.
data1.columns = data1.columns.str.strip().str.lower().str.replace('.', '_') #more convenient "_" than "."
data1['measure_time']=data1['measure_time'].astype('str') #convert to string to slice it to only Hours and minutes
data1.measure_time=data1.measure_time.str[:16]

# Update the old data with the new data for the model.
data1.update(data2)

# Export the data
export_csv=data1.to_csv('/Users/juanfatri/Documents/Kschool/00_TFM/data1.csv',index=None, header=True)

In [4]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 1 columns):
Initializing the table    0 non-null object
dtypes: object(1)
memory usage: 80.0+ bytes


In [5]:
data2.tail()

,Initializing the table


In [6]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11953 entries, 11 to 11963
Data columns (total 58 columns):
measure_time                            11953 non-null object
minute_counter                          11953 non-null object
val01_ambient_temperature               11953 non-null object
val01_ambient_winddir                   11953 non-null object
val01_ambient_windspeed                 11953 non-null object
val01_controller_hubtemperature         11953 non-null object
val01_controller_toptemperature         11953 non-null object
val01_electrical_activepower            11953 non-null object
val01_electrical_cosphi                 11953 non-null object
val01_electrical_currentphase1          11953 non-null object
val01_electrical_currentphase2          11953 non-null object
val01_electrical_currentphase3          11953 non-null object
val01_electrical_frequency              11953 non-null object
val01_electrical_reactivepower          11953 non-null object
val01_electrical_volta

In [7]:
data1.tail()

,measure_time,minute_counter,val01_ambient_temperature,val01_ambient_winddir,val01_ambient_windspeed,val01_controller_hubtemperature,val01_controller_toptemperature,val01_electrical_activepower,val01_electrical_cosphi,val01_electrical_currentphase1,...,val01_meteorological_windspeed,val01_nacelle_direction,val01_nacelle_temperature,val01_rotor_bladespitchangle1,val01_rotor_hydraulicoilpressure,val01_rotor_hydraulicoiltemperature,val01_rotor_rpm,val01_rotorsystem_bladespitchangle,val01_rotorsystem_rotorrpm,val01_spinner_temperature
11959,2019-03-24 23:20,7168800,10.9573,48.15,12.4,24,40,2000.15,1,1663,...,12.6833,47.9,26,7.82,201.127,37.1559,14.6969,7.01,14.8165,15
11960,2019-03-24 23:30,7169400,10.8778,53.355,10.64,23.3953,40,1996.91,1,1644.23,...,11,48.5782,25.7869,3.40967,183.952,37.2902,14.6948,1.529,14.6733,15
11961,2019-03-24 23:40,7170000,10.7042,51.5,12.9,23,40,2001.7,1,1648.65,...,11.3935,53.9,25.2183,1.67097,193.55,37.4244,14.6927,3.48,15,14.9083
11962,2019-03-24 23:50,7170600,10.9389,60.0667,11.9,23,40,1980.8,1,1656.67,...,10.2,58.9089,25,-0.656667,197.297,37.5586,14.6907,2.05667,14.6,14
11963,2019-03-25 00:00,7171200,10.9476,61.3333,12.0667,23,40,1999.02,1,1663.5,...,10.7833,59.1,25,3.85083,188.75,37.6928,14.6886,3.29333,14.7667,14


## Forecasting model

In [8]:
# Import the data modified
dataImport=pd.read_csv("~/Documents/Kschool/00_TFM/data1.csv")
#dataImport=data1

# convert the rows with data in float.
for col in dataImport.columns[1:]:
    dataImport[col]=dataImport[col].apply(pd.to_numeric,errors='ignore') 
    
# convert row to datetime for the time series analysis.
dataImport['measure_time']=pd.to_datetime(dataImport['measure_time'],format="%Y-%m-%d %H:%M:%S",errors="ignore") 

#set the measure_time as Index for the Time Series analysis
dataImport.set_index('measure_time', inplace=True) 

#select various columns to start the model
dataImport_selVar=dataImport[['val01_ambient_temperature','val01_ambient_winddir','val01_ambient_windspeed']] 

#select last 100 measures for the fitting of the model
dataImport_selVar100=dataImport_selVar.iloc[-100:] #select 100 hundred from the last 101 measures for training.

#create a variable to prevent original data to corrupt
dataImport_selVar_copy=dataImport_selVar 

# Initialize counter for the for loop to move along the columns (signals)
counterTest=0

for column in dataImport_selVar100:
    if counterTest==0: #initialize
        result01=autoArimaModelVar(dataImport_selVar100[column],dataImport_selVar[column],
                             dataImport_selVar_copy[column],counterTest)
    else:
        result11=autoArimaModelVar(dataImport_selVar100[column],dataImport_selVar[column],
                             dataImport_selVar_copy[column],counterTest)
        result01=result01.append(result11,sort=False) #print only 18
        
    counterTest +=1


# change the name of the NaN to "" preparing the output for Tableau.
result01NoNaN = result01.replace(np.nan, '', regex=True)

# Export the output
export_csv=result01NoNaN.to_csv('/Users/juanfatri/Documents/Kschool/00_TFM/result01NoNaN.csv',index=None, header=True)

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=156.900, BIC=169.230, Fit time=1.022 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=178.271, BIC=183.202, Fit time=0.058 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=171.791, BIC=181.654, Fit time=0.237 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=154.947, BIC=164.810, Fit time=0.724 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=154.341, BIC=166.671, Fit time=1.283 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 0, 12); AIC=171.890, BIC=181.753, Fit time=0.226 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=176.665, BIC=184.062, Fit time=0.088 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(2, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=156.2

In [9]:
result01NoNaN

,val01_ambient_temperature,Predicted,LowerConfidenceInterval,UpperConfidenceInterval,std_dev,signal,value,val01_ambient_winddir,val01_ambient_windspeed
2019-01-01 00:00:00,10.75,,,,,val01_ambient_temperature,10.75,,
2019-01-01 00:10:00,10,,,,,val01_ambient_temperature,10,,
2019-01-01 00:20:00,10,,,,,val01_ambient_temperature,10,,
2019-01-01 00:30:00,10,,,,,val01_ambient_temperature,10,,
2019-01-01 00:40:00,10,,,,,val01_ambient_temperature,10,,
2019-01-01 00:50:00,10,,,,,val01_ambient_temperature,10,,
2019-01-01 01:00:00,10,,,,,val01_ambient_temperature,10,,
2019-01-01 01:10:00,10,,,,,val01_ambient_temperature,10,,
2019-01-01 01:20:00,10,,,,,val01_ambient_temperature,10,,
2019-01-01 01:30:00,10,,,,,val01_ambient_temperature,10,,


In [10]:
result01NoNaN.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35874 entries, 2019-01-01 00:00:00 to 2019-03-25 00:50:00
Data columns (total 9 columns):
val01_ambient_temperature    35874 non-null object
Predicted                    35874 non-null object
LowerConfidenceInterval      35874 non-null object
UpperConfidenceInterval      35874 non-null object
std_dev                      35874 non-null object
signal                       35874 non-null object
value                        35874 non-null object
val01_ambient_winddir        35874 non-null object
val01_ambient_windspeed      35874 non-null object
dtypes: object(9)
memory usage: 2.7+ MB
